<a href="https://colab.research.google.com/github/crissyw/migratoryPaths/blob/master/musicbrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install arrow

     |████████████████████████████████| 51kB 1.6MB/s 


In [4]:
import pandas as pd
import requests
import json
from datetime import datetime
from pandas.io.json import json_normalize
from google.colab import auth
import time
import requests
# need to mount Google Drive below to left hand side and point to bigquery_creds.json for the bq read
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    '/content/drive/My Drive/Colab Notebooks/bigquery_creds.json',
)
project_id = "peak-segment-102813"

# Looking up the list of artists from the BQ table to pass through to MusicBrainz
artistMBIDs = """
SELECT distinct(artist_mbid) as artist_mbid
FROM `peak-segment-102813.migratorypaths.festival_tours`
"""

uniqueArtists = pandas_gbq.read_gbq(artistMBIDs, project_id=project_id, credentials=credentials)

In [6]:
i=0
for x in uniqueArtists['artist_mbid']:
  i += 1

print(i)
print(type(uniqueArtists))
print(type(uniqueArtists['artist_mbid']))

for x in uniqueArtists['artist_mbid'][:5]:
  print(x)

298
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
e02ea2a6-b58e-4b6c-8538-5b1cf3bf2fe8
b0311291-19f9-4288-96be-e8a5da0a104a
ee190f6b-7d98-43ec-b924-da5f8018eca0
478028d8-4c7b-46b2-940f-ac22b4ef37c2
6e8e3717-a62d-4a3f-886d-6a4e82507930


In [19]:
# Loopy loop
# artistSearch = api + '/artist/' +artist_mbid+ '?inc=genres+tags&fmt=json'

# putting together MusicBrainz api calls
# test: http://musicbrainz.org/ws/2/artist/0039c7ae-e1a7-4a7d-9b49-0cbc716821a6?inc=genres+tags&fmt=json
api = 'http://musicbrainz.org/ws/2'
apiHeaders = {'Accept':'application/json'}
reqCount=0
genres_list = []

for artist_mbid in uniqueArtists['artist_mbid']:
  artistGenre = api + '/artist/' + artist_mbid + '?inc=genres+tags&fmt=json'
  resp = requests.get(artistGenre)
  print(resp)
  time.sleep(60/55)
  if resp.status_code != 200:
    print('Error with status code {}'.format(resp.status_code))
    time.sleep(65/55) #ratelimit is avg 1 req per sec
    continue
  jsonResp = resp.json()
  genres_list.append(
    [
    jsonResp['id'],
    json.dumps(jsonResp['genres']),
    json.dumps(jsonResp['tags']),
    jsonResp['country'],
    jsonResp['gender']
    ])

dataset = pd.DataFrame(genres_list)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
Error with status code 404
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Res

In [30]:
# dataset.sample(5)
#dataset.columns = ['artist_mbid', 'genres', 'tags', 'country', 'gender']
dataset.head 

pandas_gbq.to_gbq(dataset, 'migratorypaths.artist_genres', project_id=project_id)

"""
with artist_genres as
(
SELECT artist_mbid, genre_name
FROM `peak-segment-102813.migratorypaths.artist_genres`,
UNNEST(REGEXP_EXTRACT_ALL(genres, r'"name"\s*:\s*"(.*?)"')) genre_name WITH OFFSET)

select genre_name, count(distinct(artist_mbid))
from artist_genres
group by 1
order by 2 desc
limit 100
"""

1it [00:04,  4.74s/it]
